# Scikit Learn 방식

In [1]:
import mlflow
mlflow.__version__

'3.1.0'

In [7]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mlflow.set_tracking_uri('http://127.0.0.1:5000') # HTTP 기반 tracking uri

In [12]:
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

with mlflow.start_run():
    # 모델 학습
    clf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
    clf.fit(X_train, y_train)

    # 예측 및 평가
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    # 매트릭, 모델 로깅, mlflow 문법
    mlflow.log_param('n_estimators', 100)
    mlflow.log_param('max_depth', 3)
    mlflow.log_metric('accuracy', acc)

    mlflow.sklearn.log_model(clf, 'model')
    print('logged model accuracy', acc)

2025/06/11 15:19:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/11 15:19:34 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/11 15:19:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


logged model accuracy 1.0
🏃 View run unique-bear-168 at: http://127.0.0.1:5000/#/experiments/0/runs/4ed22ae1163844b584fd30a81adee69c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
